In [ ]:
from tqdm.notebook import tqdm
import numpy as np
from multiinstance.data.syntheticData import buildDataset

from multiinstance.utils import getBootstrapSample, estimate

from multiinstance.gradientMethod import getGlobalAlphaHat, initDS, addEsts, g1

In [ ]:
def estGlobal(ds, bootstrapLocally=True, reps=10):
    alphaHats = np.zeros(reps)
    for rep in tqdm(range(reps),total=reps,desc="getting global estimates"):
        if bootstrapLocally:
            P, U = list(zip(*[getBootstrapSample(*dsi.getBag(int(i))) for i in range(dsi.N)]))
            ps = np.concatenate(P)
            us = np.concatenate(U)
        else:
            P,U = list(zip(*[dsi.getBag(i) for i in range(ds.N)]))
            p = np.concatenate(P)
            u = np.concatenate(U)
            ps,us = getBootstrapSample(p,u)
        alphaHats[rep],_ = estimate(ps,us)
    return alphaHats

In [ ]:
def getTrueGlobal(ds):
    return ds.trueAlphas.flatten().dot(ds.numU) / ds.numU.sum()

def getEstimatedGlobal(ds, res):
    return np.sum(np.multiply(np.multiply(res["alphaHats"],
                               res["weights"][0]),
                   ds.numU.reshape((-1,1)))) / res["weights"].flatten().dot(ds.numU)

In [ ]:
absErrsLocal = []
absErrsGlobal = []
for _ in tqdm(range(100)):
    dsi = buildDataset(100)

    dsi.globalAlphaHatsBLocally = estGlobal(dsi,bootstrapLocally=True, reps=10)
    dsi.globalAlphaHatsBGlobally = estGlobal(dsi,bootstrapLocally=False, reps=10)

    absErrsLocal.append(np.abs(dsi.globalAlphaHatsBLocally - getTrueGlobal(dsi)))

    absErrsGlobal.append(np.abs(dsi.globalAlphaHatsBGlobally - getTrueGlobal(dsi)))

In [ ]:
absErrsLocal = np.concatenate(absErrsLocal)
absErrsGlobal = np.concatenate(absErrsGlobal)

In [ ]:
np.mean(absErrsLocal), np.mean(absErrsGlobal)

In [ ]:
from scipy.stats import ks_2samp

In [ ]:
ks_2samp(absErrsLocal, absErrsGlobal)